# T1. Reading dataset(s) and viewing the (meta)data at different detail levels

## Teaching Notebook 1 (of 6) for *Intro to the NCAS CF Data Tools, cf-python and cf-plot*

**In this section we look at a basic use of cf-python, reading in one or more datasets from file and inspecting the data and the metadata at different levels of detail to suit the amount of information you want to see.**

***

## Context and learning objectives

### What are the NCAS CF Data Tools and why do they all have 'cf' in the name?

The _NCAS CF Data Tools_ are a suite of complementary Python libraries which are designed to facilitate working with data for research in the earth sciences and aligned domains. The two that are of most relevance to the average user, and those wanting to process, analyse and visualise atmospheric data, are *cf-python* (https://ncas-cms.github.io/cf-python/) and *cf-plot* (https://ncas-cms.github.io/cf-plot/build/). We will be focusing on use of cf-python and cf-plot today.

The 'cf' in the names of the NCAS CF Data Tools corresponds to the _CF Conventions_, a metadata standard, because they are built around this standard in the form of using the CF Data Model, which as well as performance is considered a 'unique selling point' of the tools.


### What are the CF Conventions?

The _CF Conventions_, usually referred to in this way but also know by the full name of the **C**limate and **F**orecast (CF) metadata conventions, are a metadata standard which is becoming the de-facto convention to cover the description of geoscientific data so that sharing and intercomparison is simpler. See https://cfconventions.org/ for more information.


### What are we going to learn in this session?

Our **learning aim** is to be able to use the NCAS CF Data Tools Python libraries, namely cf-python and cf-plot to process, analyse and visualise netCDF and PP datasets, whilst appreciating the context and 'unique selling point' of the libraries as being built to use the CF Conventions, a metadata standard for earth science data, to make it simpler to do what you want to do with the datasets, by working on top of a Data Model for CF.

We have **six distinct objectives**, matching the sections in this notebook and in the practical notebook you will work through. By the end of this lesson you should be familiar and have practiced using cf-python and cf-plot to:

1. read dataset(s) and view the (meta)data at different detail levels;
2. edit the (meta)data and write out the edited version to file;
3. reduce datasets by subspacing and collapsing;
4. visualise datasets as contour and vector plots;
5. analyse data: applying mathematical and statistical operations and plotting trends;
6. change the underlying grid of data through regridding.

<div class="alert alert-block alert-info">
<i>Note:</i> much of what you can do with cf-python you can do with the xarray library. Use whichever approach, the cf-python/cf-plot way, or the xarray way, works best for you! However, we want to emphasise that the NCAS CF Data Tools are built around the CF Conventions whereas xarray is not, so cf-python and cf-plot offer better metadata awareness to xarray, which could be a core advantage to our approach for users in/from geoscience. (If you have suggestions for how we can improve cf-python and/or cf-plot for you or your work, please let us know through the Issue Trackers linked at the end of this Notebook.)
</div>

***

## Where to find more information and resources on the NCAS CF Data Tools

Here are some links relating to the NCAS CF Data Tools and this training. The **first two are the official documentation pages** which could be useful to consult which doing the practicals, if you would like further information or if you get stuck:

* **The cf-python documentation lives at https://ncas-cms.github.io/cf-python/.**
* **The cf-plot documentation lives at https://ncas-cms.github.io/cf-plot/build/.**
* This training, with further material, is hosted online and there are instructions for setting up the environment so you can work through it in your own time: https://github.com/NCAS-CMS/cf-tools-training.
* The cf-python code lives on GitHub at https://github.com/NCAS-CMS/cf-python. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-python/issues.
* The cf-plot code lives on GitHub at https://github.com/NCAS-CMS/cf-plot. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-plot/issues.
* There is a technical presentation about the NCAS CF Data Tools avaialble from https://hps.vi4io.org/_media/events/2020/summer-school-cfnetcdf.pdf.
* The website of the CF Conventions can be found at https://cfconventions.org/.
* The landing page for training into the CF Conventions is found here within the website above: https://cfconventions.org/Training/.

If you have any queries after this course (or during, if not being taught by us in-person where you can ask us questions there and then), please either use the Issue Trackers linked above or you can email Sadie at: sadie.bartholomew@ncas.ac.uk.

***

## Setting up

**In this section we set up this Notebook, import the libraries and check the data we will work with, ready to use the libraries within this notebook.**

Run some set up for nice outputs in this Jupyter Notebook (not required in interactive Python or a script):

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

Import cf-python and cf-plot:

In [2]:
import cfplot as cfp
import cf

Inspect the versions of cf-python and cf-plot and the version of the CF Conventions those are matched to:

In [3]:
print("cf-python version is:", cf.__version__)
print("cf-plot version is:", cfp.__version__)
print("CF Conventions version is:", cf.CF())

cf-python version is: 3.18.1
cf-plot version is: 3.4.0
CF Conventions version is: 1.12


<div class="alert alert-block alert-info">
<i>Note:</i> you can work with data compliant by any other version of the CF Conventions, or without (much) compliance, but the CF Conventions version gives the maximum version that these versions of the tools understand the features of.
</div>

Finally, see what datasets we have to explore:

<div class="alert alert-block alert-info">
<i>Note:</i> in a Jupyter Notebook, '!' precedes a shell command, so this is a terminal command and not Python
</div>

In [4]:
!ls ../ncas_data

160by320griddata.nc			   precip_2010.nc
aaaaoa.pmh8dec.pp			   precip_DJF_means.nc
alpine_precip_DJF_means.nc		   qbo.nc
data1.nc				   regions.nc
data1-updated.nc			   rgp.nc
data2.nc				   sea_currents_backup.nc
data3.nc				   sea_currents.nc
data5.nc				   ta.nc
ggas2014121200_00-18.nc			   tripolar.nc
IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc  two_fields.nc
land.nc					   ua.nc
model_precip_DJF_means_low_res.nc	   u_n216.nc
model_precip_DJF_means.nc		   u_n96.nc
n2o_emissions.nc			   vaAMIPlcd_DJF.nc
POLCOMS_WAM_ZUV_01_16012006.nc		   va.nc
precip_1D_monthly.nc			   wapAMIPlcd_DJF.nc
precip_1D_yearly.nc


***

## 1. Reading dataset(s) and viewing the (meta)data at different detail levels

<div class="alert alert-block alert-info">
<i>Note:</i> In cf-python and when discussing related code and datasets, we use terminology from the CF Data Model (for more detail see: <a href="url">https://ncas-cms.github.io/cf-python/cf_data_model.html</a>). For example cf-python methods are named in relation to concepts from this data model. We don't have time to cover this in detail but for this session it is useful to know the following terms:

<ul>
    <li><b>field</b>: a self-contained cf-python object corresponding to a netCDF data variable with all of its (CF) metadata attached;</li>
    <li><b>field list</b>: a list of lields (see above), stored as its own cf-python object 'FieldList' which is similar to a Python list;</li>
    <li><b>coordinate</b>: a (CF) metadata concept which corresponds to netCDF coordinate variables. One or more coordinates are defined on every field as either 'dimension' or 'auxiliary' coordinate objects in cf-python.</li>
</ul>
</div>

The examples from this section should help you to familiarise yourself with these terms and their practical usage.

### a) Reading in data and extracting the _field_ of interest

Read a chosen data file. Sometimes datasets have descriptive names but this one doesn't, so let's find out what it is!

In [5]:
fieldlist = cf.read("../ncas_data/data1.nc")

See the 'fieldlist' that cf-python interprets from the data read in:

In [6]:
fieldlist

[<CF Field: long_name=Potential vorticity(time(1), pressure(23), latitude(160), longitude(320)) K m**2 kg**-1 s**-1>,
 <CF Field: air_temperature(time(1), pressure(23), latitude(160), longitude(320)) K>,
 <CF Field: eastward_wind(time(1), pressure(23), latitude(160), longitude(320)) m s**-1>,
 <CF Field: northward_wind(time(1), pressure(23), latitude(160), longitude(320)) m s**-1>]

Select a particular field from the fieldlist of interest:

In [7]:
field = fieldlist[0]

### b) Inspecting the _field_ of interest with different amounts of detail

View the field with **minimal detail**, i.e. a one-line summary:

In [8]:
field

<CF Field: long_name=Potential vorticity(time(1), pressure(23), latitude(160), longitude(320)) K m**2 kg**-1 s**-1>

Or you can view it with a **medium level of detail** with the Python built-in `print` function:

In [9]:
print(field)

Field: long_name=Potential vorticity (ncvar%PV)
-----------------------------------------------
Data            : long_name=Potential vorticity(time(1), pressure(23), latitude(160), longitude(320)) K m**2 kg**-1 s**-1
Dimension coords: time(1) = [1964-01-21 00:00:00]
                : pressure(23) = [1000.0, ..., 1.0] mbar
                : latitude(160) = [89.14151763916016, ..., -89.14151763916016] degrees_north
                : longitude(320) = [0.0, ..., 358.875] degrees_east


A final option is to view it with **maximal detail** using the `dump()` method:

In [10]:
field.dump()

-----------------------------------------------
Field: long_name=Potential vorticity (ncvar%PV)
-----------------------------------------------
Conventions = 'CF-1.7'
_FillValue = np.float32(2e+20)
date = '21/01/64'
history = 'Sun Sep 16 11:26:16 BST 2012 - CONVSH V1.92 16-February-2006'
long_name = 'Potential vorticity'
missing_value = np.float32(2e+20)
name = 'PV'
source = 'GRIB data'
time = '00:00'
title = 'Potential vorticity'
units = 'K m**2 kg**-1 s**-1'
valid_max = np.float32(0.018913519)
valid_min = np.float32(-0.008174051)

Data(time(1), pressure(23), latitude(160), longitude(320)) = [[[[1.3371172826737165e-06, ..., -0.0072057610377669334]]]] K m**2 kg**-1 s**-1

Domain Axis: latitude(160)
Domain Axis: longitude(320)
Domain Axis: pressure(23)
Domain Axis: time(1)

Dimension coordinate: time
    long_name = 't'
    standard_name = 'time'
    time_origin = '21-JAN-1964:00:00:00'
    units = 'days since 1964-01-21 00:00:00'
    Data(time(1)) = [1964-01-21 00:00:00]

Dimension coo

### c) Inspecting a metadata _construct_ e.g. _coordinate_ from the _field_ of interest

Use the same approach to view a particular metadata aspect, for example the latitude coordinate:

In [11]:
lat = field.coordinate("latitude")

In [12]:
lat

<CF DimensionCoordinate: latitude(160) degrees_north>

In [13]:
print(lat)

latitude(160) degrees_north


In [14]:
lat.dump()

Dimension coordinate: latitude
    long_name = 'latitude'
    standard_name = 'latitude'
    units = 'degrees_north'
    Data(160) = [89.14151763916016, ..., -89.14151763916016] degrees_north


### d) Inspecting a data array of interest

Likewise, the same approach works to view the data itself in the field (i.e. the underlying arrays). First, grab the data from the field with the `data` attribute:

In [15]:
data = field.data

Then view it in a chosen level of detail as with the above objects:

In [16]:
data

<CF Data(1, 23, 160, 320): [[[[1.3371172826737165e-06, ..., -0.0072057610377669334]]]] K m**2 kg**-1 s**-1>

In [17]:
print(data)

[[[[1.3371172826737165e-06, ..., -0.0072057610377669334]]]] K m**2 kg**-1 s**-1


In [18]:
data.dump()

Data.shape = (1, 23, 160, 320)
Data.first_datum = 1.3371172826737165e-06
Data.last_datum  = -0.0072057610377669334
Data.fill_value = np.float32(2e+20)
Data.Units = <Units: K m**2 kg**-1 s**-1>


If you want to see more of the data array itself, you can access it with the `array` attribute. Beware, for real-life datasets:

* this will be large and Python will likely truncate it so your screen isn't spammed with sub-arrays of values!
* it is computationally intensive to access the underlying data array if it is large, especially if it is multi-dimensional, so your computer will often have to work hard to get the array, so use the `array` method sparingly (only when needed)!

In [19]:
data.array

array([[[[ 1.33711728e-06,  1.34895674e-06,  1.36079620e-06, ...,
           1.33303115e-06,  1.33439312e-06,  1.33575520e-06],
         [ 9.01258318e-07,  9.05623892e-07,  9.09989467e-07, ...,
           8.74191755e-07,  8.83213943e-07,  8.92236130e-07],
         [ 1.61837670e-06,  1.61586217e-06,  1.61334754e-06, ...,
           1.54293957e-06,  1.56808528e-06,  1.59323099e-06],
         ...,
         [-4.24336758e-06, -4.21298319e-06, -4.18259879e-06, ...,
          -4.79468736e-06, -4.61091395e-06, -4.42714099e-06],
         [-4.64383629e-06, -4.58431896e-06, -4.52480162e-06, ...,
          -4.65955236e-06, -4.65431367e-06, -4.64907498e-06],
         [-1.37489405e-06, -1.36284507e-06, -1.35079608e-06, ...,
          -1.37898019e-06, -1.37761822e-06, -1.37625614e-06]],

        [[ 8.86357157e-07,  8.86566681e-07,  8.86776263e-07, ...,
           8.84785550e-07,  8.85309419e-07,  8.85833288e-07],
         [ 1.20673212e-06,  1.19610922e-06,  1.18548633e-06, ...,
           1.22594065e

***